In [1]:
## !pip3 install -q -r requirements.txt
# !pipreqsnb --force

In [7]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from IPython import display

import gymnasium as gym
import numpy as np
import random

import copy

import torch
import torch.nn as nn
import tqdm.notebook as tqdm

In [8]:
from agents import ModelFreeAgent
# from aux_func import LinearAR, ExponentialAR

In [9]:
# ENV = gym.make("Acrobot-v1", render_mode="rgb_array")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# ACTIONS_STR = ('-', '0', '+')

In [10]:
class PPOAgent(ModelFreeAgent):
    def __init__(self, env, aid_to_str, hidden_d=(64, 32), device=DEVICE):
        super().__init__(env=env, aid_to_str=aid_to_str)
        
        self.loss = nn.MSELoss()
        self.device = device

        # model for a value function (scalar)
        self.V_model = nn.Sequential(
            nn.Linear(in_features=self.d_states, out_features=hidden_d[0]),
            nn.ReLU(),
            nn.Linear(in_features=hidden_d[0], out_features=hidden_d[1]),
            nn.ReLU(),
            nn.Linear(hidden_d[1], 1))
        
        # predicts parameters mu, std of action distribution (implying that's a Normal d.)
        self.pi_model = nn.Sequential(
            nn.Linear(in_features=self.d_states, out_features=hidden_d[0]),
            nn.ReLU(),
            nn.Linear(in_features=hidden_d[0], out_features=hidden_d[1]),
            nn.ReLU(),
            nn.Linear(hidden_d[1], 2 * self.n_actions)
            nn.Tanh())

        self.fly = self.dq_step     

    def act(self, state):
        # use current model to get (epsilon-greedy) action distribution for next action
        
        action_d = self.gi(self.Q(state).squeeze().numpy(), self.curr_eps)
        return np.random.choice(self.n_actions, p=action_d)